In [1]:
pip install torch torchvision tqdm


Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm


In [3]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define transforms for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for the models
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load dataset
dataset = datasets.ImageFolder(root='/kaggle/input/gallbladder/Gallblader Diseases Dataset', transform=transform)

# Split dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [4]:
# Load the pretrained VGG16 model
model = models.vgg16(pretrained=True)

# Freeze parameters
for param in model.parameters():
    param.requires_grad = False

# Modify the classifier
num_classes = len(dataset.classes)  # Get number of classes
model.classifier[6] = nn.Linear(4096, num_classes)

# Move model to device
# model = model.to(device)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 183MB/s]


In [5]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)  # Only optimize classifier


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
import torch

# Check if multiple GPUs are available
num_gpus = torch.cuda.device_count()
print(f'Number of GPUs available: {num_gpus}')


Number of GPUs available: 2


In [8]:
# Assuming 'model' is your VGG16 model
if num_gpus > 1:
    model = nn.DataParallel(model)

# Move the model to the GPU(s)
model = model.to(device)


In [9]:
import time
from tqdm import tqdm

def train_model(model, data_loader, criterion, optimizer, num_epochs=5):
    model.train()
    
    for epoch in range(num_epochs):
        start_time = time.time()  # Start timing for the epoch
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        
        # Create a progress bar for the training loop
        with tqdm(total=len(data_loader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
            for inputs, labels in data_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # Backward pass and optimization
                loss.backward()
                optimizer.step()
                
                # Calculate loss
                running_loss += loss.item()
                
                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total_predictions += labels.size(0)
                correct_predictions += (predicted == labels).sum().item()
                
                # Update progress bar
                pbar.update(1)

            # Calculate average loss and accuracy for the epoch
            epoch_loss = running_loss / len(data_loader)
            epoch_accuracy = correct_predictions / total_predictions * 100
            
            # Calculate time taken for the epoch
            epoch_time = time.time() - start_time
            
            # Print loss, accuracy, and time taken for the epoch
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Time: {epoch_time:.2f} seconds')

# Example usage
# train_model(model, train_loader, criterion, optimizer, num_epochs=5)


# Example usage
train_model(model, train_loader, criterion, optimizer, num_epochs=10)


Epoch 1/10:   0%|          | 0/268 [00:00<?, ?batch/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
Epoch 1/10: 100%|██████████| 268/268 [03:39<00:00,  1.22batch/s]


Epoch [1/10], Loss: 1.7347, Accuracy: 37.25%, Time: 219.47 seconds


Epoch 2/10: 100%|██████████| 268/268 [02:42<00:00,  1.65batch/s]


Epoch [2/10], Loss: 1.4272, Accuracy: 49.94%, Time: 162.18 seconds


Epoch 3/10: 100%|██████████| 268/268 [02:43<00:00,  1.64batch/s]


...

Epoch [8/10], Loss: 1.1716, Accuracy: 58.42%, Time: 162.08 seconds


Epoch 9/10: 100%|██████████| 268/268 [02:42<00:00,  1.65batch/s]


Epoch [9/10], Loss: 1.1253, Accuracy: 60.66%, Time: 162.46 seconds


Epoch 10/10: 100%|██████████| 268/268 [02:41<00:00,  1.66batch/s]

Epoch [10/10], Loss: 1.1411, Accuracy: 59.67%, Time: 161.28 seconds


In [10]:
def test_model(model, data_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # Disable gradient calculation for testing
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Calculate loss
            running_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
            total_predictions += labels.size(0)  # Total number of labels
            correct_predictions += (predicted == labels).sum().item()  # Count correct predictions

    # Calculate average loss and accuracy for the test set
    test_loss = running_loss / len(data_loader)
    test_accuracy = correct_predictions / total_predictions * 100  # Convert to percentage
    
    # Print test results
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Example usage
# Assuming you have a DataLoader for your test dataset
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
test_model(model, test_loader, criterion)


Test Loss: 0.8057, Test Accuracy: 75.18%


In [11]:
import torch

# Assuming 'model' is your trained model
torch.save(model, 'vgg16_model_gallbladder.pth')
